# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import random

In [3]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import BernoulliNB, GaussianNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.neural_network import MLPClassifier, MLPRegressor
# from sklearn.svm import SVC
# from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

In [4]:
# from imblearn.under_sampling import RandomUnderSampler

In [5]:
# from imblearn.over_sampling import SMOTE, RandomOverSampler
# from imblearn.pipeline import Pipeline

In [6]:
from collections import Counter

In [7]:
import wandb

In [8]:
# from sklearn import preprocessing

In [9]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

In [10]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EXS

In [11]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F

In [12]:
import torch.optim as optim

In [13]:
torch.__version__

'1.2.0'

In [14]:
from tqdm.notebook import tqdm

In [15]:
import math

# Function

In [16]:
# Function(Utility)

# check Duplicate file
def checkDuplicateFile(file_path):
    import os
    if os.path.isfile(file_path):
        print("Caution: File existed!")
        ans = input("Do you want to cover it?(Y/others)")
        if ans == "Y":
            return False
        else:
            print("Canceled....")
            return True
    else:
        return False


# Function(Use)

# Test for checking ICU_id missing in Lab_1103_csv
def getMissingIDinLab(Lab_file, show=True):
    test = sorted(Lab_file.ICU_id.unique())
    s = 0
    error_list = list()
    for i in test:
        #         print(i)
        s += 1
        if s != i:
            if show:
                print(f"error! : {s}")
            error_list.append(s)
            s += 1
    if show:
        print(f"Missing ID Result: {error_list}")
    if show:
        print(f"Missing Length:{len(error_list)}")
    return error_list

# store Dataframe to CSV


def store2CSV(data, target_name, target_loc_prefix='./'):
    file_path = target_loc_prefix+target_name+".csv"
    if checkDuplicateFile(file_path):
        print("store2CSV failed")
        return
    data.to_csv(file_path)
    print("store2CSV Successful!")

# store Datastruc. to pickle


def store2Pickle(data, target_name, target_loc_prefix='./'):
    import pickle
    file_path = target_loc_prefix+target_name+'.pickle'
    if checkDuplicateFile(file_path):
        print("store2Pickle failed")
        return
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)
    print("store2Pickle Successful!")


def readFPickle(target_name, target_loc_prefix='./'):
    import pickle
    file_name = target_loc_prefix+target_name+'.pickle'
    with open(file_name, 'rb') as f:
        temp = pickle.load(f)
    return temp

# Function(Data preprocessing)

#Function: 補值
# 將針對輸入的df_data直接進行inplace插補
# 須確保df_data的缺失值位置有放np.nan


def handleMissing(df_data, df_feature, outFeature=["outcome"], cate_astype="int"):
    for featureName in df_data.columns:
        if featureName not in outFeature:
            if df_data[featureName].isna().sum() == 0:
                print(f"{featureName}: Not need to fill.")
                continue
            else:
                # 先去看是連續與否 (1代表連續,0代表離散)
                kindValue = df_feature.loc[df_feature["features name"]
                                           == featureName, "kind"].values[0]
                if kindValue == 1:
                    # continuous
                    # mean filling
                    targetMean = df_data[featureName].mean()
                    df_data[featureName].fillna(value=targetMean, inplace=True)
                    print(f"{featureName}: Fill, Continuous.")

                else:
                    # categorical
                    # mode filling
                    targetMode = df_data[featureName].mode()[0]
                    df_data[featureName].fillna(value=targetMode, inplace=True)
                    df_data[featureName] = df_data[featureName].astype(
                        cate_astype)
                    print(
                        f"{featureName}: Fill, Categorical. (astype to {cate_astype})")

    print("---handleMissing Finish---")

# plot hist
# filtered_data need to check not have nan


def plotHist(df_data, target, outcome="outcome", bins=20):
    filtered_data = pd.concat([df_data["outcome"], df_data[target]], axis=1)
    filtered_data = filtered_data.dropna()
    print(filtered_data.isna().sum())
    plt.hist(filtered_data.loc[filtered_data[outcome]
             == 0, target], bins=bins, alpha=0.5, label='0')
    plt.hist(filtered_data.loc[filtered_data[outcome]
             == 1, target], bins=bins, alpha=0.5, label='1')
    plt.xlabel(target)
    plt.ylabel('count')
    plt.legend(title=outcome)

# plot countplot
# filtered_data need to check not have nan


def plotCountplot(df_data, target, outcome="outcome"):
    filtered_data = pd.concat([df_data["outcome"], df_data[target]], axis=1)
    filtered_data = filtered_data.dropna()
    print(filtered_data.isna().sum())
    sns.countplot(x=target, hue=outcome, data=filtered_data)

# plot boxplot
# filtered_data need to check not have nan


def plotBoxplot(df_data, target, outcome="outcome"):
    filtered_data = pd.concat([df_data["outcome"], df_data[target]], axis=1)
    filtered_data = filtered_data.dropna()
    print(filtered_data.isna().sum())
    sns.boxplot(x=filtered_data[target], data=filtered_data)

# Function(Model)


def serializeModel(model, modelName, featureNum):
    initial_type = [('float_input', FloatTensorType([None, featureNum]))]
    onx = convert_sklearn(model, initial_types=initial_type)
    with open(modelName + ".onnx", "wb") as f:
        f.write(onx.SerializeToString())


def modelPredict(modelName, testData):
    sess = rt.InferenceSession(modelName + '.onnx')  # load the onnx
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    pred_onx = sess.run([label_name], {input_name: testData.astype(np.float32)})[
        0]  # predict testData
    print(pred_onx)

In [17]:
def get_data(X, y):
    X = torch.from_numpy(X).float()#transform the data from numpy to torch
    y = torch.from_numpy(y).long()
    
    dataset = TensorDataset(X,y)
    
    return dataset


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         num_workers=2, pin_memory=True)
    return loader

In [18]:
# ConvNet(self, kernel_size->list, input_size, classes=2)
def make(config, X_train, y_train, X_val, y_val):
    # Make the data
    # X_train, y_train, X_val, y_val is dataframe
    # now to ndarray
    X_train = X_train.values
    y_train = y_train.values
    X_val = X_val.values
    y_val = y_val.values
    train, test = get_data(X_train, y_train), get_data(X_val, y_val)
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    kernel_size = [config.kernel_size_1, config.kernel_size_2]
    fc_layer_size = [config.fc_layer_size_1, config.fc_layer_size_2]
    # Make the model
    model = ConvNet(kernel_size, config.feature_size , fc_layer_size, config.classes).to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    if config.optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)
        print("-->Using SGD")
    if config.optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(),lr=config.learning_rate)
        print("-->Using Adam")

    return model, train_loader, test_loader, criterion, optimizer

# model_pipeline

In [19]:
def model_pipeline(hyperparameters, X_train, y_train, X_val, y_val, run_name="CNN"):

    # tell wandb to get started
    with wandb.init(project="DataMining_Project2", entity="oscarchencs10", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config
        
        wandb.run.name = run_name
        wandb.run.save()

        # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer = make(config, X_train, y_train, X_val, y_val)
        print(model)

        # and use them to train the model
        train(model, train_loader, test_loader,criterion, optimizer, config)

        # and test its final performance
        test(model, test_loader)

    return model

# model_pipeline_for sweap

In [20]:
def model_pipeline_sweap(hyperparameters=None):

    # tell wandb to get started
    with wandb.init(project="DataMining_Project2", entity="oscarchencs10", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config
        
        wandb.run.name = config.run_name
        wandb.run.save()
        
        X_train = exp13_X_train
        y_train = exp13_y_train
        X_val = exp13_X_val
        y_val = exp13_y_val
        
        # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer = make(config, X_train, y_train, X_val, y_val)
        print(model)

        # and use them to train the model
        train(model, train_loader, test_loader,criterion, optimizer, config)

        # and test its final performance
        test(model, test_loader)

    return model

# Train_log

In [21]:
# {"acc": correct / total, "f1":f1Score, "auc":auc}
def train_log(loss, example_ct, epoch, metrics, now_batch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss, "training_acc":metrics["acc"], 
               "training_f1":metrics["f1"], "training_auc":metrics["auc"]}, step=example_ct)
    print(f"----Epoch[{epoch}],Batch[{now_batch}]----")
    acc = metrics["acc"]
    f1 = metrics["f1"]
    auc = metrics["auc"]
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")
    print(f"Acc after " + str(example_ct).zfill(5) + f" examples: {acc:.4f}")
    print(f"f1_score after " + str(example_ct).zfill(5) + f" examples: {f1:.4f}")
    print(f"AUC after " + str(example_ct).zfill(5) + f" examples: {auc:.4f}")
    print(f"<-------------------------------------->")

# Model Architecture

In [22]:
# Conventional and convolutional neural network

class ConvNet(nn.Module):
    def __init__(self, kernel_size, input_size, fc_layer_size, classes=2):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=kernel_size[0]),
            nn.BatchNorm1d(6, eps=1e-05),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=1),)
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=6, out_channels=16, kernel_size=kernel_size[1]),
            nn.BatchNorm1d(16, eps=1e-05),
            nn.ReLU(),
#             nn.Dropout(p=0.2),
            nn.MaxPool1d(kernel_size=1),)
        
        self.flatten = nn.Flatten()
        
        self.fc = nn.Sequential(
            nn.Linear(16*(input_size-kernel_size[0]+1-kernel_size[1]+1), fc_layer_size[0]),
            nn.ReLU(),
            nn.Linear(fc_layer_size[0], fc_layer_size[1]),
            nn.ReLU(),
            nn.Linear(fc_layer_size[1], classes),
        )
        
    def forward(self, x):
#         print(x.shape)
        x = torch.reshape(x, (-1,1, 40))
#         print(x.shape)
        out = self.layer1(x)
#         print(out.shape)
        out = self.layer2(out)
        out = self.flatten(out)
#         print(out.shape)
        out = self.fc(out)
#         print(out.shape)
        return out

# Train/Test

In [23]:
# {"acc": correct / total, "f1":f1_score, "auc":auc}
def train(model, loader, test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        now_batch = 0
        for _, (data_row, labels) in enumerate(loader):

            loss = train_batch(data_row, labels, model, optimizer, criterion)
            example_ct +=  len(data_row)
            batch_ct += 1
            now_batch += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 100) == 0:
                metrics = test(model, test_loader, training_phase=True)
                train_log(loss, example_ct, epoch, metrics, now_batch)


def train_batch(data_row, labels, model, optimizer, criterion):
    data_row, labels = data_row.to(device), labels.to(device)
    model.train()
    # Forward pass ➡
    outputs = model(data_row)
    loss = criterion(outputs, labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [24]:
def test(model, test_loader, training_phase=False):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        prob_all = []
        prob_all2 = []
        label_all = []
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1) #predicted:預測標籤結果
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prob_all2.extend(outputs[:,1].cpu().numpy())
            prob_all.extend(predicted.cpu().numpy())
            label_all.extend(labels.cpu().numpy())

        f1Score = f1_score(label_all, prob_all)
        auc = roc_auc_score(label_all,prob_all2)
        print(f"Accuracy of the model on the {total} " +
              f"test data: {100 * correct / total}%")
        print(f"F1-Score of the model on the {total} " +
              "test data:{:.4f}".format(f1Score))
        print(f"AUC of the model on the {total} " +
              "test data:{:.4f}".format(auc))
        
        if not training_phase:
            wandb.log({"test_accuracy": correct / total, "test_f1":f1Score, "test_auc":auc})
            return None
        else:
            return {"acc": correct / total, "f1":f1Score, "auc":auc}

#     # Save the model in the exchangeable ONNX format
#     torch.onnx.export(model, images, "model.onnx")
#     wandb.save("model.onnx")

# Load Data

In [25]:
exp13_X_train = readFPickle("data/exp13/exp13_X_train")
exp13_X_train

,AGE,SEX,LOS,Joint,Drain,Cemented,Commercial_ALBC,Non_commercial_ALBC,cci_index,elx_index,...,Coagulopathy,Fluid and Electrolyte Disorders,Blood Loss Anemia,Deficiency Anemia,Anemia,Alcohol Abuse,Drug Abuse,Psychoses,Depression,Psyciatric disorder
25753,66,0,6,0,1,1,1,0,4,3,...,0,0,0,0,0,0,0,0,1,1
628,61,0,10,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26083,80,1,2,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37780,67,0,6,0,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
28632,62,0,4,1,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,70,0,7,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
44732,72,0,3,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
38158,68,0,6,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
860,31,1,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
exp13_y_train = readFPickle("data/exp13/exp13_y_train")
exp13_y_train

25753    0
628      0
26083    0
37780    1
28632    0
        ..
11284    0
44732    1
38158    1
860      0
15795    0
Name: outcome, Length: 45627, dtype: int64

In [27]:
exp13_X_val = readFPickle("data/exp13/exp13_X_val")
exp13_X_val

,AGE,SEX,LOS,Joint,Drain,Cemented,Commercial_ALBC,Non_commercial_ALBC,cci_index,elx_index,...,Coagulopathy,Fluid and Electrolyte Disorders,Blood Loss Anemia,Deficiency Anemia,Anemia,Alcohol Abuse,Drug Abuse,Psychoses,Depression,Psyciatric disorder
286,72.0,0,13,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
40555,78.0,1,3,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
28125,69.0,0,8,1,1,0,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
44616,72.0,1,7,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49606,80.0,0,3,1,0,0,0,0,2,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23499,81.0,1,5,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
47449,71.0,0,7,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10480,55.0,0,3,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
35724,73.0,0,7,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
exp13_y_val = readFPickle("data/exp13/exp13_y_val")
exp13_y_val

286      1
40555    0
28125    0
44616    0
49606    0
        ..
23499    0
47449    0
10480    0
35724    0
5180     0
Name: outcome, Length: 8037, dtype: int64

# Random Seed

In [29]:
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# GPU Setting

In [30]:
# Device configuration
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [31]:
torch.cuda.current_device()

0

In [32]:
torch.cuda.device_count()

4

In [33]:
torch.cuda.get_device_name(1)

'Tesla K40m'

In [34]:
torch.cuda.set_device(1)

In [35]:
torch.cuda.current_device()

1

# Config

In [36]:
config = dict(
    epochs=100,
    classes=2,
    kernel_size_1=1,
    kernel_size_2=1,
    feature_size=40,
    batch_size=128,
    learning_rate=0.001,
    dataset="PJI",
    architecture="CNN(BN, drop)")

# Main

In [35]:
# Build, train and analyze the model with the pipeline
model = model_pipeline(config, exp13_X_train, exp13_y_train, exp13_X_val, exp13_y_val, run_name="CNN lr 0.001")

wandb: Currently logged in as: oscarchencs10 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


ConvNet(
  (layer1): Sequential(
    (0): Conv1d(1, 6, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(6, 16, kernel_size=(1,), stride=(1,))
    (1): ReLU()
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=640, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy of the model on the 8037 test data: 97.87234042553192%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6391
----Epoch[0],Batch[99]----
Loss after 12672 examples: 0.560
Acc after 12672 examples: 0.9787
f1_score after 12672 examples: 0.0000
AUC after 12672 examples: 0.6391
<-------------------------------------->
Accuracy of the model on the 8037 test data: 96.72763468956079%
F1-Score of the model on the 8037 test data:0.0436
AUC of the model on the 8037 test data:0.6571
----Epoch[0],Batch[199]----
Loss after 25472 examples: 0.648
Acc after 25472 examples: 0.9673
f1_score after 25472 examples: 0.0436
AUC after 25472 examples: 0.6571
<-------------------------------------->
Accuracy of the model on the 8037 test data: 95.93131765584174%
F1-Score of the model on the 8037 test data:0.0630
AUC of the model on the 8037 test data:0.6614
----Epoch[0],Batch[299]----
Loss after 38272 examples: 0.630
Acc after 38272 examples: 0.9593
f1_score aft

...

Accuracy of the model on the 8037 test data: 92.18613910663183%
F1-Score of the model on the 8037 test data:0.0368
AUC of the model on the 8037 test data:0.5926
----Epoch[81],Batch[82]----
Loss after 3706283 examples: 0.328
Acc after 3706283 examples: 0.9219
f1_score after 3706283 examples: 0.0368
AUC after 3706283 examples: 0.5926
<-------------------------------------->
Accuracy of the model on the 8037 test data: 95.5953714072415%
F1-Score of the model on the 8037 test data:0.0328
AUC of the model on the 8037 test data:0.6364
----Epoch[81],Batch[182]----
Loss after 3719083 examples: 0.372
Acc after 3719083 examples: 0.9560
f1_score after 3719083 examples: 0.0328
AUC after 3719083 examples: 0.6364
<-------------------------------------->
Accuracy of the model on the 8037 test data: 91.21562772178673%
F1-Score of the model on the 8037 test data:0.0711
AUC of the model on the 8037 test data:0.5860
----Epoch[81],Batch[282]----
Loss after 3731883 examples: 0.309
Acc after 3731883 example

Accuracy of the model on the 8037 test data: 95.98108747044917%
F1-Score of the model on the 8037 test data:0.0798
AUC of the model on the 8037 test data:0.5916
----Epoch[87],Batch[140]----
Loss after 3987469 examples: 0.356
Acc after 3987469 examples: 0.9598
f1_score after 3987469 examples: 0.0798
AUC after 3987469 examples: 0.5916
<-------------------------------------->
Accuracy of the model on the 8037 test data: 90.61838994649744%
F1-Score of the model on the 8037 test data:0.0916
AUC of the model on the 8037 test data:0.6046
----Epoch[87],Batch[240]----
Loss after 4000269 examples: 0.403
Acc after 4000269 examples: 0.9062
f1_score after 4000269 examples: 0.0916
AUC after 4000269 examples: 0.6046
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.07739206171458%
F1-Score of the model on the 8037 test data:0.1152
AUC of the model on the 8037 test data:0.6282
----Epoch[87],Batch[340]----
Loss after 4013069 examples: 0.447
Acc after 4013069 examp

Accuracy of the model on the 8037 test data: 93.14420803782505%
F1-Score of the model on the 8037 test data:0.0832
AUC of the model on the 8037 test data:0.6334
----Epoch[93],Batch[198]----
Loss after 4268655 examples: 0.362
Acc after 4268655 examples: 0.9314
f1_score after 4268655 examples: 0.0832
AUC after 4268655 examples: 0.6334
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.25158641284061%
F1-Score of the model on the 8037 test data:0.1217
AUC of the model on the 8037 test data:0.6677
----Epoch[93],Batch[298]----
Loss after 4281455 examples: 0.303
Acc after 4281455 examples: 0.9425
f1_score after 4281455 examples: 0.1217
AUC after 4281455 examples: 0.6677
<-------------------------------------->
Accuracy of the model on the 8037 test data: 92.06171457011322%
F1-Score of the model on the 8037 test data:0.0274
AUC of the model on the 8037 test data:0.6140
----Epoch[94],Batch[41]----
Loss after 4294186 examples: 0.304
Acc after 4294186 exampl

Accuracy of the model on the 8037 test data: 91.68844096055741%
F1-Score of the model on the 8037 test data:0.0824
AUC of the model on the 8037 test data:0.6353
----Epoch[99],Batch[256]----
Loss after 4549841 examples: 0.352
Acc after 4549841 examples: 0.9169
f1_score after 4549841 examples: 0.0824
AUC after 4549841 examples: 0.6353
<-------------------------------------->
Accuracy of the model on the 8037 test data: 87.545103894488%
F1-Score of the model on the 8037 test data:0.0974
AUC of the model on the 8037 test data:0.6660
----Epoch[99],Batch[356]----
Loss after 4562641 examples: 0.397
Acc after 4562641 examples: 0.8755
f1_score after 4562641 examples: 0.0974
AUC after 4562641 examples: 0.6660
<-------------------------------------->
Accuracy of the model on the 8037 test data: 89.43635684957073%
F1-Score of the model on the 8037 test data:0.0920
AUC of the model on the 8037 test data:0.6650


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▇▇▇▇▆▇▇█▇▇▅▇▅▄▄▅▃▄▅▃▃▄▂▅▂▅▄▃▁▃▁▃▂▁▂▂▃▁▅▂
test_accuracy,▁
test_auc,▁
test_f1,▁
training_acc,████████████▇▁▇▇▇▇▆▇▇▆▇▇▇█▅█▇▆▇▇▆█▇█▇▇▇▆
training_auc,▇▇███████▇▇▇▃▂▂▂▂▁▂▂▂▂▃▃▄▄▃▄▂▃▂▄▃▃▆▄▃▅▆▅
training_f1,▅▄▇▇▇▇▇█▆▇▇▇▄▄▃▂▁▄▅▁▁▆▆▅▅▁▆▁▄▆▂▄▅▃▆▆▂▃█▆
epoch,99
loss,0.39741
test_accuracy,0.89436


In [71]:
# Build, train and analyze the model with the pipeline
model2 = model_pipeline(config, exp13_X_train, exp13_y_train, exp13_X_val, exp13_y_val, run_name="CNN new archi(drop)")

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


ConvNet(
  (layer1): Sequential(
    (0): Conv1d(1, 6, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(6, 16, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=640, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.4141
----Epoch[0],Batch[99]----
Loss after 12672 examples: 0.610
Acc after 12672 examples: 0.9788
f1_score after 12672 examples: 0.0000
AUC after 12672 examples: 0.4141
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.4886
----Epoch[0],Batch[199]----
Loss after 25472 examples: 0.609
Acc after 25472 examples: 0.9788
f1_score after 25472 examples: 0.0000
AUC after 25472 examples: 0.4886
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5942
----Epoch[0],Batch[299]----
Loss after 38272 examples: 0.612
Acc after 38272 examples: 0.9788
f1_score aft

Accuracy of the model on the 8037 test data: 96.8147318651238%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.4849
----Epoch[6],Batch[257]----
Loss after 306658 examples: 0.471
Acc after 306658 examples: 0.9681
f1_score after 306658 examples: 0.0000
AUC after 306658 examples: 0.4849
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.2889137737962%
F1-Score of the model on the 8037 test data:0.0213
AUC of the model on the 8037 test data:0.5134
----Epoch[6],Batch[357]----
Loss after 319389 examples: 0.529
Acc after 319389 examples: 0.9429
f1_score after 319389 examples: 0.0213
AUC after 319389 examples: 0.5134
<-------------------------------------->
Accuracy of the model on the 8037 test data: 91.56401642403883%
F1-Score of the model on the 8037 test data:0.0314
AUC of the model on the 8037 test data:0.5599
----Epoch[7],Batch[100]----
Loss after 332189 examples: 0.590
Acc after 332189 examples: 0.9156
f1_

...

Accuracy of the model on the 8037 test data: 93.84098544232923%
F1-Score of the model on the 8037 test data:0.0275
AUC of the model on the 8037 test data:0.5577
----Epoch[81],Batch[82]----
Loss after 3706283 examples: 0.479
Acc after 3706283 examples: 0.9384
f1_score after 3706283 examples: 0.0275
AUC after 3706283 examples: 0.5577
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.71656090581062%
F1-Score of the model on the 8037 test data:0.0232
AUC of the model on the 8037 test data:0.5321
----Epoch[81],Batch[182]----
Loss after 3719083 examples: 0.341
Acc after 3719083 examples: 0.9372
f1_score after 3719083 examples: 0.0232
AUC after 3719083 examples: 0.5321
<-------------------------------------->
Accuracy of the model on the 8037 test data: 95.35896478785617%
F1-Score of the model on the 8037 test data:0.0210
AUC of the model on the 8037 test data:0.5452
----Epoch[81],Batch[282]----
Loss after 3731883 examples: 0.441
Acc after 3731883 exampl

Accuracy of the model on the 8037 test data: 89.78474555182282%
F1-Score of the model on the 8037 test data:0.0596
AUC of the model on the 8037 test data:0.5486
----Epoch[87],Batch[140]----
Loss after 3987469 examples: 0.341
Acc after 3987469 examples: 0.8978
f1_score after 3987469 examples: 0.0596
AUC after 3987469 examples: 0.5486
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.65434863755132%
F1-Score of the model on the 8037 test data:0.0267
AUC of the model on the 8037 test data:0.5314
----Epoch[87],Batch[240]----
Loss after 4000269 examples: 0.348
Acc after 4000269 examples: 0.9365
f1_score after 4000269 examples: 0.0267
AUC after 4000269 examples: 0.5314
<-------------------------------------->
Accuracy of the model on the 8037 test data: 95.25942515864128%
F1-Score of the model on the 8037 test data:0.0155
AUC of the model on the 8037 test data:0.5659
----Epoch[87],Batch[340]----
Loss after 4013069 examples: 0.368
Acc after 4013069 examp

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6703
----Epoch[93],Batch[198]----
Loss after 4268655 examples: 0.512
Acc after 4268655 examples: 0.9788
f1_score after 4268655 examples: 0.0000
AUC after 4268655 examples: 0.6703
<-------------------------------------->
Accuracy of the model on the 8037 test data: 95.19721289038198%
F1-Score of the model on the 8037 test data:0.0203
AUC of the model on the 8037 test data:0.5921
----Epoch[93],Batch[298]----
Loss after 4281455 examples: 0.439
Acc after 4281455 examples: 0.9520
f1_score after 4281455 examples: 0.0203
AUC after 4281455 examples: 0.5921
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.17693169092945%
F1-Score of the model on the 8037 test data:0.0751
AUC of the model on the 8037 test data:0.6023
----Epoch[94],Batch[41]----
Loss after 4294186 examples: 0.380
Acc after 4294186 exampl

Accuracy of the model on the 8037 test data: 94.74928455891502%
F1-Score of the model on the 8037 test data:0.0231
AUC of the model on the 8037 test data:0.5514
----Epoch[99],Batch[256]----
Loss after 4549841 examples: 0.318
Acc after 4549841 examples: 0.9475
f1_score after 4549841 examples: 0.0231
AUC after 4549841 examples: 0.5514
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.08983451536643%
F1-Score of the model on the 8037 test data:0.0404
AUC of the model on the 8037 test data:0.5984
----Epoch[99],Batch[356]----
Loss after 4562641 examples: 0.447
Acc after 4562641 examples: 0.9409
f1_score after 4562641 examples: 0.0404
AUC after 4562641 examples: 0.5984
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.35572974990669%
F1-Score of the model on the 8037 test data:0.0361
AUC of the model on the 8037 test data:0.5707


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▆▇▆▄▃▅▄▆▃▃▅▃▆▄▃▂▂▁▅▄▃▃▂▃▃▂▂▂▄▅▄▄▂▁▁▃▄▁
test_accuracy,▁
test_auc,▁
test_f1,▁
training_acc,██▄▇▇▆▆▅▄▃▆▅▆▇▆▂▅▄▅▆▆▆▂▁▆█▆▆▅▆▄▅▆▃▄▇▅▅▆▆
training_auc,██▅▂▁▁▂▂▂▃▁▂▂▁▂▂▃▃▃▂▃▃▃▂▄▆▃▃▃▃▄▃▄▃▂▆▆▇▃▄
training_f1,▁▁▃▂▂▃▃▄▅▆▂▃▃▂▃▅▅▆▅▃▃▂▆▅▂▁▂▃▅▃▅▃▃▄▃▂▆█▂▃
epoch,99
loss,0.44743
test_accuracy,0.93356


In [36]:
# Build, train and analyze the model with the pipeline
model3 = model_pipeline(config, exp13_X_train, exp13_y_train, exp13_X_val, exp13_y_val, run_name="CNN new archi2")

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


ConvNet(
  (layer1): Sequential(
    (0): Conv1d(1, 6, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(6, 16, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=640, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.4059
----Epoch[0],Batch[99]----
Loss after 12672 examples: 0.601
Acc after 12672 examples: 0.9788
f1_score after 12672 examples: 0.0000
AUC after 12672 examples: 0.4059
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.4477
----Epoch[0],Batch[199]----
Loss after 25472 examples: 0.620
Acc after 25472 examples: 0.9788
f1_score after 25472 examples: 0.0000
AUC after 25472 examples: 0.4477
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5216
----Epoch[0],Batch[299]----
Loss after 38272 examples: 0.648
Acc after 38272 examples: 0.9788
f1_score aft

Accuracy of the model on the 8037 test data: 95.19721289038198%
F1-Score of the model on the 8037 test data:0.0153
AUC of the model on the 8037 test data:0.6152
----Epoch[6],Batch[257]----
Loss after 306658 examples: 0.504
Acc after 306658 examples: 0.9520
f1_score after 306658 examples: 0.0153
AUC after 306658 examples: 0.6152
<-------------------------------------->
Accuracy of the model on the 8037 test data: 78.87271369914147%
F1-Score of the model on the 8037 test data:0.0567
AUC of the model on the 8037 test data:0.6139
----Epoch[6],Batch[357]----
Loss after 319389 examples: 0.569
Acc after 319389 examples: 0.7887
f1_score after 319389 examples: 0.0567
AUC after 319389 examples: 0.6139
<-------------------------------------->
Accuracy of the model on the 8037 test data: 89.06308324001493%
F1-Score of the model on the 8037 test data:0.0393
AUC of the model on the 8037 test data:0.5942
----Epoch[7],Batch[100]----
Loss after 332189 examples: 0.573
Acc after 332189 examples: 0.8906
f

...

Accuracy of the model on the 8037 test data: 94.2267015055369%
F1-Score of the model on the 8037 test data:0.0794
AUC of the model on the 8037 test data:0.6665
----Epoch[81],Batch[82]----
Loss after 3706283 examples: 0.256
Acc after 3706283 examples: 0.9423
f1_score after 3706283 examples: 0.0794
AUC after 3706283 examples: 0.6665
<-------------------------------------->
Accuracy of the model on the 8037 test data: 90.54373522458629%
F1-Score of the model on the 8037 test data:0.1080
AUC of the model on the 8037 test data:0.6570
----Epoch[81],Batch[182]----
Loss after 3719083 examples: 0.288
Acc after 3719083 examples: 0.9054
f1_score after 3719083 examples: 0.1080
AUC after 3719083 examples: 0.6570
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.55480900833645%
F1-Score of the model on the 8037 test data:0.1452
AUC of the model on the 8037 test data:0.6772
----Epoch[81],Batch[282]----
Loss after 3731883 examples: 0.240
Acc after 3731883 example

Accuracy of the model on the 8037 test data: 85.70362075401269%
F1-Score of the model on the 8037 test data:0.1044
AUC of the model on the 8037 test data:0.6995
----Epoch[87],Batch[140]----
Loss after 3987469 examples: 0.263
Acc after 3987469 examples: 0.8570
f1_score after 3987469 examples: 0.1044
AUC after 3987469 examples: 0.6995
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.4133383103148%
F1-Score of the model on the 8037 test data:0.1851
AUC of the model on the 8037 test data:0.7003
----Epoch[87],Batch[240]----
Loss after 4000269 examples: 0.264
Acc after 4000269 examples: 0.9441
f1_score after 4000269 examples: 0.1851
AUC after 4000269 examples: 0.7003
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.82854298867737%
F1-Score of the model on the 8037 test data:0.1419
AUC of the model on the 8037 test data:0.6711
----Epoch[87],Batch[340]----
Loss after 4013069 examples: 0.225
Acc after 4013069 exampl

Accuracy of the model on the 8037 test data: 96.92671394799055%
F1-Score of the model on the 8037 test data:0.2006
AUC of the model on the 8037 test data:0.7043
----Epoch[93],Batch[198]----
Loss after 4268655 examples: 0.234
Acc after 4268655 examples: 0.9693
f1_score after 4268655 examples: 0.2006
AUC after 4268655 examples: 0.7043
<-------------------------------------->
Accuracy of the model on the 8037 test data: 90.28244369789722%
F1-Score of the model on the 8037 test data:0.1274
AUC of the model on the 8037 test data:0.7008
----Epoch[93],Batch[298]----
Loss after 4281455 examples: 0.289
Acc after 4281455 examples: 0.9028
f1_score after 4281455 examples: 0.1274
AUC after 4281455 examples: 0.7008
<-------------------------------------->
Accuracy of the model on the 8037 test data: 90.54373522458629%
F1-Score of the model on the 8037 test data:0.1304
AUC of the model on the 8037 test data:0.7061
----Epoch[94],Batch[41]----
Loss after 4294186 examples: 0.285
Acc after 4294186 exampl

Accuracy of the model on the 8037 test data: 92.84558915018042%
F1-Score of the model on the 8037 test data:0.1221
AUC of the model on the 8037 test data:0.6746
----Epoch[99],Batch[256]----
Loss after 4549841 examples: 0.305
Acc after 4549841 examples: 0.9285
f1_score after 4549841 examples: 0.1221
AUC after 4549841 examples: 0.6746
<-------------------------------------->
Accuracy of the model on the 8037 test data: 96.16772427522707%
F1-Score of the model on the 8037 test data:0.1765
AUC of the model on the 8037 test data:0.7019
----Epoch[99],Batch[356]----
Loss after 4562641 examples: 0.358
Acc after 4562641 examples: 0.9617
f1_score after 4562641 examples: 0.1765
AUC after 4562641 examples: 0.7019
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.43822321761851%
F1-Score of the model on the 8037 test data:0.1738
AUC of the model on the 8037 test data:0.7047


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▆▅▅▄▅▅▆▅▂▄▃▂▂▃▂▁▁▂▄▂▄▂▃▂▃▂▂▂▃▁▁▁▄▃▂▃▂
test_accuracy,▁
test_auc,▁
test_f1,▁
training_acc,███▇▃█▁▇███▇▄▆█▇█▇█▇██▇▆▆▆▇▇▇▇▆█▇▇▇▇▇▇▇▇
training_auc,▃▆▅▂▃▂▃▂▂▂▁▁▃▄▄▄▅▅▅▆▆▇▄▅▇▆▆▇▇▇▇█▆▇▇▇██▇▆
training_f1,▁▁▁▂▃▁▃▃▁▁▁▃▃▄▁▄▃▅▄▅▅▆▃▄▅▅▆▇▆▆▅█▆▆▆▆▆▆▆▅
epoch,99
loss,0.35799
test_accuracy,0.94438


# Sweap

In [37]:
sweep_config = {
    'method': 'random'
    }

In [38]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
    'fc_layer_size_1': {
        'values': [32, 120, 128, 256]
    },
    'fc_layer_size_2': {
        'values': [32, 120, 128, 256]
    },
    'kernel_size_1': {
        'values': [1, 5, 10]
    },
    'kernel_size_2': {
        'values': [1, 5, 10, 15, 20]
    },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
    },
    'epochs': {
        'value': 50
    },
    'classes': {
        'value': 2
    },
    'feature_size': {
        'value': 40
    },
    'dataset': {
        'value': "PJI"
    },
    'architecture': {
        'value': "CNN(BN)"
    },
    'run_name': {
        'value': "CNN(Sweap)"
    },
}

sweep_config['parameters'] = parameters_dict

In [39]:
sweep_id = wandb.sweep(sweep_config, project="DataMining_Project2")

Create sweep with ID: i4taydrx
Sweep URL: https://wandb.ai/oscarchencs10/DataMining_Project2/sweeps/i4taydrx


In [40]:
wandb.agent(sweep_id, model_pipeline_sweap, count=1)

wandb: Agent Starting Run: d5oca0zh with config:
wandb: 	architecture: CNN(BN)
wandb: 	batch_size: 108
wandb: 	classes: 2
wandb: 	dataset: PJI
wandb: 	epochs: 50
wandb: 	fc_layer_size_1: 32
wandb: 	fc_layer_size_2: 32
wandb: 	feature_size: 40
wandb: 	kernel_size_1: 1
wandb: 	kernel_size_2: 1
wandb: 	learning_rate: 0.0363598893586223
wandb: 	optimizer: sgd
wandb: 	run_name: CNN(Sweap)
wandb: Currently logged in as: oscarchencs10 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


-->Using SGD
ConvNet(
  (layer1): Sequential(
    (0): Conv1d(1, 6, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(6, 16, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=640, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)


  0%|          | 0/50 [00:00<?, ?it/s]

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6653
----Epoch[0],Batch[99]----
Loss after 10692 examples: 0.592
Acc after 10692 examples: 0.9788
f1_score after 10692 examples: 0.0000
AUC after 10692 examples: 0.6653
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6759
----Epoch[0],Batch[199]----
Loss after 21492 examples: 0.600
Acc after 21492 examples: 0.9788
f1_score after 21492 examples: 0.0000
AUC after 21492 examples: 0.6759
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6744
----Epoch[0],Batch[299]----
Loss after 32292 examples: 0.567
Acc after 32292 examples: 0.9788
f1_score aft

...

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[32],Batch[163]----
Loss after 1477668 examples: 0.550
Acc after 1477668 examples: 0.9788
f1_score after 1477668 examples: 0.0000
AUC after 1477668 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[32],Batch[263]----
Loss after 1488468 examples: 0.638
Acc after 1488468 examples: 0.9788
f1_score after 1488468 examples: 0.0000
AUC after 1488468 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[32],Batch[363]----
Loss after 1499268 examples: 0.579
Acc after 1499268 examp

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[37],Batch[248]----
Loss after 1714983 examples: 0.561
Acc after 1714983 examples: 0.9788
f1_score after 1714983 examples: 0.0000
AUC after 1714983 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[37],Batch[348]----
Loss after 1725783 examples: 0.616
Acc after 1725783 examples: 0.9788
f1_score after 1725783 examples: 0.0000
AUC after 1725783 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[38],Batch[25]----
Loss after 1736526 examples: 0.601
Acc after 1736526 exampl

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[42],Batch[333]----
Loss after 1952298 examples: 0.695
Acc after 1952298 examples: 0.9788
f1_score after 1952298 examples: 0.0000
AUC after 1952298 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[43],Batch[10]----
Loss after 1963041 examples: 0.758
Acc after 1963041 examples: 0.9788
f1_score after 1963041 examples: 0.0000
AUC after 1963041 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[43],Batch[110]----
Loss after 1973841 examples: 0.591
Acc after 1973841 exampl

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[47],Batch[418]----
Loss after 2189613 examples: 0.623
Acc after 2189613 examples: 0.9788
f1_score after 2189613 examples: 0.0000
AUC after 2189613 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[48],Batch[95]----
Loss after 2200356 examples: 0.601
Acc after 2200356 examples: 0.9788
f1_score after 2200356 examples: 0.0000
AUC after 2200356 examples: 0.5000
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.5000
----Epoch[48],Batch[195]----
Loss after 2211156 examples: 0.616
Acc after 2211156 exampl

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▃▁▂▃▄▄▁▅▃▁▃▁▂▃▃▄▃▄▆▄▅▃▃▄▄▂▃▆▄▄▂▃▂▄█▅▆▄▃▃
test_accuracy,▁
test_auc,▁
test_f1,▁
training_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_auc,█▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,49
loss,0.67446
test_accuracy,0.97885


# Sweap-2

In [37]:
sweep_config = {
    'method': 'random'
    }

In [38]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
    'fc_layer_size_1': {
        'values': [32, 120, 128, 256]
    },
    'fc_layer_size_2': {
        'values': [32, 120, 128, 256]
    },
    'kernel_size_1': {
        'values': [1, 5, 10]
    },
    'kernel_size_2': {
        'values': [1, 5, 10, 15, 20]
    },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
    },
    'epochs': {
        'value': 50
    },
    'classes': {
        'value': 2
    },
    'feature_size': {
        'value': 40
    },
    'dataset': {
        'value': "PJI"
    },
    'architecture': {
        'value': "CNN(BN)"
    },
    'run_name': {
        'value': "CNN(Sweap)"
    },
}

sweep_config['parameters'] = parameters_dict

In [39]:
sweep_id = wandb.sweep(sweep_config, project="DataMining_Project2")

Create sweep with ID: 4k7a8ejx
Sweep URL: https://wandb.ai/oscarchencs10/DataMining_Project2/sweeps/4k7a8ejx


In [40]:
wandb.agent(sweep_id, model_pipeline_sweap, count=5)

wandb: Agent Starting Run: bj7mfjxy with config:
wandb: 	architecture: CNN(BN)
wandb: 	batch_size: 38
wandb: 	classes: 2
wandb: 	dataset: PJI
wandb: 	epochs: 50
wandb: 	fc_layer_size_1: 120
wandb: 	fc_layer_size_2: 256
wandb: 	feature_size: 40
wandb: 	kernel_size_1: 1
wandb: 	kernel_size_2: 10
wandb: 	learning_rate: 0.08153063912136153
wandb: 	optimizer: sgd
wandb: 	run_name: CNN(Sweap)
wandb: Currently logged in as: oscarchencs10 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


-->Using SGD
ConvNet(
  (layer1): Sequential(
    (0): Conv1d(1, 6, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(6, 16, kernel_size=(10,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=496, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2, bias=True)
  )
)


  0%|          | 0/50 [00:00<?, ?it/s]

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6851
----Epoch[0],Batch[99]----
Loss after 03762 examples: 0.653
Acc after 03762 examples: 0.9788
f1_score after 03762 examples: 0.0000
AUC after 03762 examples: 0.6851
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.40089585666293%
F1-Score of the model on the 8037 test data:0.0385
AUC of the model on the 8037 test data:0.6736
----Epoch[0],Batch[199]----
Loss after 07562 examples: 0.637
Acc after 07562 examples: 0.9440
f1_score after 07562 examples: 0.0385
AUC after 07562 examples: 0.6736
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6795
----Epoch[0],Batch[299]----
Loss after 11362 examples: 0.681
Acc after 11362 examples: 0.9788
f1_score aft

wandb: Ctrl + C detected. Stopping sweep.
